In [1]:
import os
import shutil
import cv2

# Extract frames

In [2]:
root_dir = '/Users/inaki-eab/Desktop/test_data'
videos_dir = os.path.join(root_dir, 'videos')
frames_dir = os.path.join(root_dir,'video_frames')
images_dir = os.path.join(root_dir,'images')

In [ ]:
def extract_frames(video_path, output_folder, frame_rate):
    # Extract the video filename without the extension
    video_filename = os.path.splitext(os.path.basename(video_path))[0]

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Create a subfolder for the frames using the video filename
    os.makedirs(output_folder, exist_ok=True)

    # Get the frames per second (FPS) of the video
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Calculate the frame interval based on the desired frame rate
    frame_interval = fps // frame_rate

    # Initialize variables
    frame_count = 0
    success = True

    # Loop through the video and extract frames
    while success:
        success, frame = cap.read()
        # Only save frames at the desired frame rate
        if frame_count % frame_interval == 0:
            if success:
                frame_filename = os.path.join(output_folder, f"{video_filename}_frame_{frame_count:04d}.jpg")
                cv2.imwrite(frame_filename, frame)
            else:
                break
        frame_count += 1

    # Release the video file
    cap.release()

In [ ]:
for file in os.listdir(videos_dir):
    if file.endswith('.mp4'):
        print(file)
        extract_frames(os.path.join(videos_dir, file), frames_dir, 2)

# Rename frames and images

In [ ]:
import re

def rename_dir(dir, shift=1):
    files = [f for f in os.listdir(dir)]
    #files.sort(key=lambda x: int(re.match(r'\d+', x).group()))
    
    for idx, old_filename in enumerate(files):
        extension = os.path.splitext(old_filename)[1]
        new_filename = f'{idx+shift}{extension}'
        old_path = os.path.join(dir, old_filename)
        new_path = os.path.join(dir, new_filename)
        os.rename(old_path, new_path)

In [ ]:
# Start with video frames
old_len = len(os.listdir(frames_dir))
print(old_len)
rename_dir(frames_dir)
print(len(os.listdir(frames_dir)))
assert old_len == len(os.listdir(frames_dir))

In [ ]:
# Continue with images
old_len = len(os.listdir(images_dir))
print(old_len)
rename_dir(images_dir, 361)
print(len(os.listdir(images_dir)))
assert old_len == len(os.listdir(images_dir))

# ALREADY ANNOTATED

### Creating a new empty dataset

In [2]:
root_path = '/Users/inaki-eab/Desktop/datasets'
#root_path = '/data-fast/127-data2/ierregue/datasets'

dataset_name = 'custom_client'

new_dataset_root = os.path.join(root_path, dataset_name)

# Create folder structure
if not os.path.isdir(new_dataset_root):
    os.makedirs(new_dataset_root)
    os.makedirs(os.path.join(new_dataset_root, 'labels','val'))
    os.makedirs(os.path.join(new_dataset_root, 'images','val'))

In [3]:
wanted_indices = [
    5, #drone/uav
    0, #aeroplane
    1, #boat
    2, #car
    3, #person
    4, #truck
]

In [4]:
def clean_map_file(file):
# Mapping should be performed only once
    
    class_index_map = {
        5:3, # drone/uav
        0:4, # aeroplane
        1:5, # boat
        2:1, # car
        3:0, # person
        4:2, # truck
    }
    
    with open(file, 'r+') as fp:
        # read an store all lines into list
        lines = fp.readlines()
        # move file pointer to the beginning of a file
        fp.seek(0)
        # truncate the file
        fp.truncate()
    
        # start writing lines
        # iterate line and line number
        for number, line in enumerate(lines):
            # Only write rows of interested instances
            old_class_id = int(line.split()[0])
            if old_class_id in wanted_indices:
                # map old class indices to new ones
                new_class_id = class_index_map[old_class_id]
                new_line_splitted = line.split()
                new_line_splitted[0] = str(new_class_id)
                new_line = ' '.join(new_line_splitted)
                fp.write(new_line+'\n')

In [5]:
def move_desired_files(
    original_dataset_root, #../datasets/old_dataset
    target_dataset_root, # ../datasets/new_dataset
    original_dataset_slice, # liketrain,test,val
    target_dataset_slice, # train,test,val
    wanted_indices # list of desired indices
):
    # Empty list to store the selected files containing at list one of the desired objects
    selected_images = []
    
    
    original_labels_dir = os.path.join(original_dataset_root, 'labels', original_dataset_slice)
    original_images_dir = os.path.join(original_dataset_root, 'images', original_dataset_slice)

    # Iterate over all files in the original dataset labels folder
    for filename in os.listdir(original_labels_dir):
        if filename.endswith('.txt'):
            # Read file
            with open(os.path.join(original_labels_dir, filename), "r") as f:
                # Empty list to store objects/instances present in image
                indices_in_file = []
                # Iterate over instances in image and get present class ids
                for line in f:
                    indices_in_file.append(int(line.split()[0]))
                # If any present class ids is a class id of interest, get its filename
                if any((True for x in indices_in_file if x in wanted_indices)):
                    # Get only name, no '.txt' extension
                    selected_images.append(os.path.splitext(filename)[0])
                    # Copy *.txt folder
                    shutil.copy(os.path.join(original_labels_dir, filename), os.path.join(target_dataset_root, 'labels', target_dataset_slice))
                    # Copy *jpg image
                    img_path = os.path.join(original_images_dir, os.path.splitext(filename)[0]+'.jpg')
                    shutil.copy(img_path, os.path.join(target_dataset_root, 'images', target_dataset_slice))

                    # Map old index to new one and delete unwanted instances
                    clean_map_file(os.path.join(target_dataset_root, 'labels', target_dataset_slice, filename))
                    
    return selected_images

In [6]:
original_dataset_path = 'client_samples'
original_dataset_root = os.path.join(root_path, original_dataset_path)

In [7]:
test_indices = move_desired_files(original_dataset_root, 
                                   new_dataset_root, 
                                   'val', 
                                   'val', 
                                   wanted_indices)

### Rename dataset files

In [8]:
old_len_img_test = len(os.listdir(os.path.join(new_dataset_root,'images','val')))
old_len_txt_test = len(os.listdir(os.path.join(new_dataset_root,'labels','val')))

In [10]:
import re

def rename_dir(dir, shift=1):
    files = [f for f in os.listdir(dir)]
    files.sort(key=lambda x: int(re.match(r'\d+', x).group()))
    #files.sort()
    for idx, old_filename in enumerate(files):
        extension = os.path.splitext(old_filename)[1]
        new_filename = f'{idx+shift}{extension}'
        old_path = os.path.join(dir, old_filename)
        new_path = os.path.join(dir, new_filename)
        os.rename(old_path, new_path)

In [11]:
rename_dir(os.path.join(new_dataset_root,'images','val'), shift=1+10617)
rename_dir(os.path.join(new_dataset_root,'labels','val'), shift=1+10617)

In [12]:
assert old_len_img_test == len(os.listdir(os.path.join(new_dataset_root,'images','val')))
assert old_len_txt_test == len(os.listdir(os.path.join(new_dataset_root,'labels','val')))

### Inspecting the new dataset

In [13]:
import pandas as pd
import imagesize
import matplotlib.pyplot as plt

In [14]:
class_names = {
    0:'person',
    1:'car', 
    2:'truck', 
    3:'uav', 
    4:'airplane', 
    5:'ship'
}

In [15]:
def inspect_dataset(
    target_dataset_root, # ../datasets/new_dataset
    target_dataset_slice, # train,test,val
):
    info = [] #list of lists, each list corresponds to an instance [cls_id, x, y, w, h, img]
    
    
    target_labels_dir = os.path.join(target_dataset_root, 'labels', target_dataset_slice)

    # Iterate over all files in the original dataset labels folder
    for filename in os.listdir(target_labels_dir):
        if filename.endswith('.txt'):
            # Read file
            with open(os.path.join(target_labels_dir, filename), "r") as f:
                # Iterate over instances in image and get present class ids
                for line in f:
                    line_data = []
                    # label data
                    line_data = line.split()
                    # Image name
                    line_data.append(os.path.splitext(filename)[0][:5])
                    # Image size: could be done at image level and not row level
                    img_path = os.path.join(target_dataset_root,'images',target_dataset_slice, os.path.splitext(filename)[0]+'.jpg')
                    img_w, img_h = imagesize.get(img_path)
                    line_data.extend([img_w, img_h])
                    # Append line data to info
                    info.append(line_data)

    df = pd.DataFrame(info, columns=['new_class_id', 'xcn', 'ycn', 'wn', 'hn', 'img', 'img_w', 'img_h'])
    df = df.astype('float32').astype({'new_class_id':'int32', 'img':'int64'})
    df['class_name'] = df['new_class_id'].map(class_names)
    return df

In [16]:
df = inspect_dataset(new_dataset_root, 'test')
df

In [17]:
print(f"The number of objects is {len(df)}")
print(f"The number of images is {len(df['img'].unique())}")

In [18]:
# Create dir to store plots
save_dir = './data/dataset_creation'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

In [20]:
fig = plt.figure(figsize=(8, 6))
ax = df['class_name'].value_counts().plot(kind='bar')
# Set x-axis label
ax.set_xlabel("Classes", weight='bold', size=12)
# Set y-axis label
ax.set_ylabel("Counts", weight='bold', size=12)
ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,3))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
None
fig.savefig(save_dir+'/client_class_counts.png', bbox_inches = 'tight')

In [21]:
fig = plt.figure(figsize=(8, 6))
ax = df.groupby(by=['img'])['img'].count().value_counts().sort_index().plot(kind='bar')
# Set x-axis label
ax.set_xlabel("Instances per image", weight='bold', size=12)
# Set y-axis label
ax.set_ylabel("Counts", weight='bold', size=12)
ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,3))
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
None
fig.savefig(save_dir+'/client_instances_count.png', bbox_inches = 'tight')

In [23]:
df['bbox_area'] = (df['wn']*df['img_w'])*(df['hn']*df['img_h'])
bin_edges = [0, 16**2, 32**2, 96**2, float('inf')]
bin_labels = ['tiny', 'small', 'medium', 'large']
df['bbox_size_category'] = pd.cut(df['bbox_area'], bins=bin_edges, labels=bin_labels, right=False)

In [24]:
fig = plt.figure(figsize=(8, 6))
ax = df['bbox_size_category'].value_counts().sort_index().plot(kind='bar')
# Set x-axis label
ax.set_xlabel("Object category size", weight='bold', size=12, labelpad=10)
# Set y-axis label
ax.set_ylabel("Counts", weight='bold', size=12)
ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,3))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
None
fig.savefig(save_dir+'/client_objsz_counts.png', bbox_inches = 'tight')

In [25]:
# TAKE INTO ACCOUNT THAT IMAGE IS UHD, THUS OBJECTS HAVE MORE PIXELS
df['bbox_area'] = (df['wn']*df['img_w'])*(df['hn']*df['img_h'])
bin_edges = [i * (df['img_w'].mean()*df['img_h'].mean())/(640*480) for i in bin_edges]
bin_labels = ['tiny', 'small', 'medium', 'large']
df['bbox_size_category_hd'] = pd.cut(df['bbox_area'], bins=bin_edges, labels=bin_labels, right=False)

In [26]:
fig = plt.figure(figsize=(8, 6))
ax = df['bbox_size_category_hd'].value_counts().sort_index().plot(kind='bar')
# Set x-axis label
ax.set_xlabel("Object category size", weight='bold', size=12, labelpad=10)
# Set y-axis label
ax.set_ylabel("Counts", weight='bold', size=12)
ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,3))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
None
fig.savefig(save_dir+'/client_objszhd_counts.png', bbox_inches = 'tight')